In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True

In [8]:
num_epochs = 100
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5
split_indices = (df_rumours[df_rumours['set']=='train'].index,
                 df_rumours[df_rumours['set']=='dev'].index,
                 df_rumours[df_rumours['set']=='test'].index)

## UMAP

In [9]:
size = 35
swmhau_network_umap, best_swmhau_network_umap, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_35_best_model.csv


In [10]:
swmhau_network_umap.groupby(["dimensions",
                             "output_channels",
                             "sig_depth",
                             "num_heads",
                             "num_layers",
                             "ffn_hidden_dim",
                             "learning_rate"]).mean()

/tmp/ipykernel_1298102/3742016164.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap.groupby(["dimensions",


loss  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (128, 128)     0.0001         0.416103   
                                                                         0.0005         0.322329   
                                                                         0.0010         0.357057   
                                                          (256, 256)     0.0001         0.484244   
                                                                         0.0005         0.300911   
                                                                         0.0010         0.323981   
                                                          (512, 512)     0.0001         0.475924   
                                                                         0.0005         0.328637   
                                                                         0.0010         0.328253   
                                     12        1          (128, 128)     0.0001         0.410756   
                                                                         0.0005         0.327853   
                                                                         0.0010         0.313763   
                                                          (256, 256)     0.0001         0.402339   
                                                                         0.0005         0.323641   
                                                                         0.0010         0.328126   
                                                          (512, 512)     0.0001         0.442708   
                                                                         0.0005         0.328185   
                                                                         0.0010         0.329972   
           12              3         10        1          (128, 128)     0.0001         0.376536   
                                                                         0.0005         0.398366   
                                                                         0.0010         0.416268   
                                                          (256, 256)     0.0001         0.604803   
                                                                         0.0005         0.469330   
                                                                         0.0010         0.435206   
                                                          (512, 512)     0.0001         0.624987   
                                                                         0.0005         0.610253   
                                                                         0.0010         0.427063   

                                                                                        accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (128, 128)     0.0001         0.632189   
                                                                         0.0005         0.633302   
                                                                         0.0010         0.626152   
                                                          (256, 256)     0.0001         0.625040   
                                                                         0.0005         0.632984   
                                                                         0.0010         0.609628   
                                                          (512, 512)     0.0001         0.614871   
                                                                         0.0005         0.626946   
                                                                         0.0010         0.616301   
                                     12        1          (128, 128)     0.0001         0.6

In [11]:
best_swmhau_network_umap

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.360839,0.628217,0.626408,"[0.6004098360655737, 0.6524064171122995]",0.637304,"[0.6861826697892272, 0.5884244372990354]",0.632849,"[0.5336976320582878, 0.732]",0.213752,0.701068,...,0.0005,1,focal,2,False,None,Conv1d,None,concatenation,64
0,0.648381,0.604385,0.603750,"[0.5878848063555113, 0.6196150320806599]",0.609100,"[0.6462882096069869, 0.571912013536379]",0.607581,"[0.5391621129326047, 0.676]",0.243665,0.772242,...,0.0005,12,focal,2,False,None,Conv1d,None,concatenation,64
0,0.831684,0.589133,0.589109,"[0.5859750240153698, 0.5922421948912014]",0.590929,"[0.6199186991869918, 0.5619389587073609]",0.590778,"[0.5555555555555556, 0.626]",0.639707,0.697509,...,0.0005,123,focal,2,False,None,Conv1d,None,concatenation,64


In [12]:
best_swmhau_network_umap[["dimensions",
                          "output_channels",
                          "sig_depth",
                          "num_heads",
                          "num_layers",
                          "ffn_hidden_dim",
                          "dropout_rate",
                          "learning_rate"]]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(512, 512)",0.1,0.0005
0,15,12,3,10,1,"(512, 512)",0.1,0.0005
0,15,12,3,10,1,"(512, 512)",0.1,0.0005


In [13]:
best_swmhau_network_umap["f1"].mean()

0.6064222184201026

In [14]:
best_swmhau_network_umap["precision"].mean()

0.6124441646876636

In [15]:
best_swmhau_network_umap["recall"].mean()

0.6104025500910747

In [16]:
np.stack(best_swmhau_network_umap["f1_scores"]).mean(axis=0)

array([0.59142322, 0.62142121])

In [17]:
np.stack(best_swmhau_network_umap["precision_scores"]).mean(axis=0)

array([0.65079653, 0.5740918 ])

In [18]:
np.stack(best_swmhau_network_umap["recall_scores"]).mean(axis=0)

array([0.5428051, 0.678    ])

## Random Projections

In [19]:
size = 35
swmhau_network_grp, best_swmhau_network_grp, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=df_rumours,
    id_column="timeline_id",
    label_column="label",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    device=device,
    split_indices=split_indices,
    k_fold=False,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_grp_focal_2_35_best_model.csv


In [20]:
swmhau_network_grp.groupby(["dimensions",
                            "output_channels",
                            "sig_depth",
                            "num_heads",
                            "num_layers",
                            "ffn_hidden_dim",
                            "dropout_rate",
                            "learning_rate"]).mean()

/tmp/ipykernel_1298102/3837375781.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_grp.groupby(["dimensions",


loss  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (128, 128)     0.1          0.0001         0.329374   
                                                                                      0.0005         0.439792   
                                                                                      0.0010         0.315147   
                                                                         0.5          0.0001         0.463705   
                                                                                      0.0005         0.350249   
                                                                                      0.0010         0.374076   
                                                          (256, 256)     0.1          0.0001         0.483176   
                                                                                      0.0005         0.368917   
                                                                                      0.0010         0.354118   
                                                                         0.5          0.0001         0.580154   
                                                                                      0.0005         0.636260   
                                                                                      0.0010         0.331611   
                                                          (512, 512)     0.1          0.0001         0.457884   
                                                                                      0.0005         0.374147   
                                                                                      0.0010         0.335981   
                                                                         0.5          0.0001         0.368866   
                                                                                      0.0005         0.388800   
                                                                                      0.0010         0.328409   
                                     12        1          (128, 128)     0.1          0.0001         0.369603   
                                                                                      0.0005         0.428862   
                                                                                      0.0010         0.319800   
                                                                         0.5          0.0001         0.391020   
                                                                                      0.0005         0.417029   
                                                                                      0.0010         0.519513   
                                                          (256, 256)     0.1          0.0001         0.456234   
                                                                                      0.0005         0.486402   
                                                                                      0.0010         0.332204   
                                                                         0.5          0.0001         0.458592   
                                                                                      0.0005         0.416198   
                                                                                      0.0010         0.336515   
                                                          (512, 512)     0.1          0.0001         0.480149   
                                                                                      0.0005         0.448931   
                                                                                      0.0010         0.365532   
                                                                         0.5          0.0001         0.347221   
                                     

In [21]:
best_swmhau_network_grp

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,0.358502,0.617731,0.617563,"[0.6095423563777994, 0.6255835667600375]",0.620751,"[0.6548117154811716, 0.5866900175131349]",0.620064,"[0.5701275045537341, 0.67]",0.209083,0.718861,...,0.0001,1,focal,2,False,None,Conv1d,None,concatenation,64
0,0.413685,0.594852,0.590673,"[0.632034632034632, 0.5493107104984094]",0.593480,"[0.6023102310231023, 0.5846501128668171]",0.591423,"[0.6648451730418944, 0.518]",0.168258,0.811388,...,0.0001,12,focal,2,False,None,Conv1d,None,concatenation,64
0,0.618928,0.563394,0.563304,"[0.5570599613152806, 0.5695488721804511]",0.565524,"[0.5938144329896907, 0.5372340425531915]",0.565295,"[0.5245901639344263, 0.606]",0.213818,0.829181,...,0.0001,123,focal,2,False,None,Conv1d,None,concatenation,64


In [22]:
best_swmhau_network_grp.columns

Index(['loss', 'accuracy', 'f1', 'f1_scores', 'precision', 'precision_scores',
       'recall', 'recall_scores', 'valid_loss', 'valid_accuracy', 'valid_f1',
       'valid_f1_scores', 'valid_precision', 'valid_precision_scores',
       'valid_recall', 'valid_recall_scores', 'k', 'dimensions', 'sig_depth',
       'method', 'input_channels', 'output_channels', 'features',
       'standardise_method', 'include_features_in_path', 'embedding_dim',
       'num_features', 'log_signature', 'num_heads', 'num_layers',
       'ffn_hidden_dim', 'dropout_rate', 'learning_rate', 'seed',
       'loss_function', 'gamma', 'k_fold', 'n_splits', 'augmentation_type',
       'hidden_dim_aug', 'comb_method', 'batch_size'],
      dtype='object')

In [23]:
best_swmhau_network_grp[["dimensions",
                         "output_channels",
                         "sig_depth",
                         "num_heads",
                         "num_layers",
                         "ffn_hidden_dim",
                         "dropout_rate",
                         "learning_rate"]]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,8,4,6,1,"(128, 128)",0.5,0.0001
0,15,8,4,6,1,"(128, 128)",0.5,0.0001
0,15,8,4,6,1,"(128, 128)",0.5,0.0001


In [24]:
best_swmhau_network_grp["f1"].mean()

0.5905133498611016

In [25]:
best_swmhau_network_grp["precision"].mean()

0.5932517587378513

In [26]:
best_swmhau_network_grp["recall"].mean()

0.5922604735883424

In [27]:
np.stack(best_swmhau_network_grp["f1_scores"]).mean(axis=0)

array([0.59954565, 0.58148105])

In [28]:
np.stack(best_swmhau_network_grp["precision_scores"]).mean(axis=0)

array([0.61697879, 0.56952472])

In [29]:
np.stack(best_swmhau_network_grp["recall_scores"]).mean(axis=0)

array([0.58652095, 0.598     ])